# Bathymetric particle filter 

This notebook is a testing and development environment for the geophysical particle filter. As of 30 December 2024 the initial implementation will be a velocity-state based implementation that does not do the proposed full state mechanization. The full state mechanization will be implemented in the future under a ROS2 and Gazebo simulation environment that allows for more realistic (and reliable) IMU simulation.

This notebook is meant for interactive testing and development and should not be used for large-scale simulations. The intended workflow is to use the notebook to test and develop the particle filter on a small subsample of test data, then build and install it in the local virtual environment for use in the full simulation.

This full simulation should then be written in the `/scripts` folder with result data saved off there.

## Simulation parameters verification

First we need to tune the particle filter propagation noise to be similar to that of a marine-grade inertial navigation system. A low-end marine-grade INS should have a drift of 1 nm per 24 hours. Our typical time interval is 60 seconds and the noise parameters for velocities states need to be tuned based on this value.

In [10]:
from src.geophysical.particle_filter import propagate_ned, rmse
import numpy as np


def run_velocity_loop(u: np.ndarray, noise: np.ndarray, dt: float = 60.0, time: int = 24 * 60) -> float:
    P = np.asarray([[0, 0, 0, u[0], u[1], u[2]]])
    T = P.copy()
    t = 0
    while t < time:
        P = propagate_ned(P, u, dt)  # noise=np.diag(noise), noise_calibration_mode=True)
        # Add zero-mean noise to the particles
        P += np.random.multivariate_normal(np.zeros(6), np.diag(noise))
        T = propagate_ned(T, u, dt)  # noise=np.diag([0, 0, 0]), noise_calibration_mode=False)
        t += 1
    error = rmse(P, T, include_altitude=True)
    return error

In [47]:
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
time = 24 * 60  # minutes
vel_noise = 1.75  # m/s
noise = np.array([0, 0, 0, vel_noise, vel_noise, 0.01])
bound = 1852  # meters
errors = []
errors_dict = {}
for i in range(1, 26):  # 1 to 25 nautical miles per hour
    print(f"Running for {i} knots")
    v = i * bound / 3600  # convert to m/s
    for a in tqdm(range(1000)):
        # Eastward
        u = np.asarray([0.0, v, 0.0])
        errors.append(run_velocity_loop(u, noise))
        # Northward
        u = np.asarray([v, 0, 0])
        errors.append(run_velocity_loop(u, noise))
        # Northeastward
        u = np.array([1, 1, 0]) / np.linalg.norm([1, 1, 0])
        u *= v
        errors.append(run_velocity_loop(u, noise))
    errors_dict[i] = errors.copy()
    print(f"Mean RMSE: {np.mean(errors): 0.2f} meters")
print(f"Overall RMSE: {np.mean(errors)}")
# Plot the previous

plt.figure(figsize=(12, 4))
plt.violinplot(errors_dict.values(), showmeans=True, showmedians=True)
plt.axhline(y=1852, color="k", linestyle="--", label="1 Nautical Mile")
plt.axhline(np.mean(errors), color="r", linestyle="-", label="Mean Error")
plt.xlabel("Velocity (knots)")
plt.ylabel("Error (meters)")
plt.title("Velocity Noise Tuning Error Distributions")
plt.legend(loc="upper left")
plt.savefig("velocity_noise_tuning.png")
plt.show()

Finalized noise model: $\begin{bmatrix} 0 & 0 & 0 & 1.75 & 1.75 & 0.01 \end{bmatrix}$

---------

## Develop measurement model

Next we need to develop the measurement value standard deviation. We'll first do some general examination of the data. Namely, investigating the sensor measurements to see if we can build a reasonable sensor model.

In [49]:
from src.data_management import dbmgr

In [50]:
db = dbmgr.DatabaseManager(".db")
summary = db.get_all_trajectories()
summary = summary.drop(summary.index[-1])
bathy_trajectories = summary[summary["depth"]]

In [ ]:
from src.geophysical import gmt_toolbox as gmt
from tqdm import tqdm

bathy_differences = np.empty((0,))

for id in tqdm(bathy_trajectories["id"]):
    data = db.get_trajectory(id)
    min_lon = data["lon"].min()
    max_lon = data["lon"].max()
    min_lat = data["lat"].min()
    max_lat = data["lat"].max()
    # min_lon, min_lat, max_lon, max_lat = gmt.inflate_bounds(min_lon, min_lat, max_lon, max_lat, 0.25)
    try:
        bathy_map = gmt.GeophysicalMap(
            gmt.MeasurementType.BATHYMETRY,
            gmt.ReliefResolution.FIFTEEN_SECONDS,
            min_lon,
            max_lon,
            min_lat,
            max_lat,
            0.25,
        )
    except AssertionError:
        if min_lat == max_lat:
            min_lat -= 0.25
            max_lat += 0.25
        if min_lon == max_lon:
            min_lon -= 0.25
            max_lon += 0.25
        bathy_map = gmt.GeophysicalMap(
            gmt.MeasurementType.BATHYMETRY,
            gmt.ReliefResolution.FIFTEEN_SECONDS,
            min_lon,
            max_lon,
            min_lat,
            max_lat,
            0.1,
        )
    except:
        print(f"Failed to get bathymetry map for {id}: {min_lon}, {max_lon}, {min_lat}, {max_lat}")
    # d_bathy = np.hstack([d_bathy, data["DEPTH"] - (-get_map_point(bathy_map, data.LON, data.LAT))])
    bathy_differences = np.append(
        bathy_differences, data["depth"] - (-bathy_map.get_map_point(data["lon"], data["lat"]))
    )

bathy_mean_d = np.mean(bathy_differences, where=~np.isnan(bathy_differences))
bathy_std = np.std(bathy_differences, where=~np.isnan(bathy_differences))

In [ ]:
from scipy.stats import mode


def plot_measurement_statistics(
    data: np.ndarray, title: str, xlabel: str, ylabel: str, filename: str, bin_count: int = 1000, xlims: tuple = None
):
    Mean = np.mean(data, where=~np.isnan(data))
    Std = np.std(data, where=~np.isnan(data))
    Median = np.median(data[~np.isnan(data)])
    Mode = mode(data[~np.isnan(data)]).mode
    plt.figure(figsize=(8, 4))
    plt.hist(data, bins=bin_count, density=True, alpha=0.75)
    plt.axvline(Mean, color="r", linestyle="--", label=f"Mean={Mean:0.2f}")
    plt.axvline(Mean + Std, color="k", linestyle="--", label=f"$\pm\sigma={Std:0.2f}$")
    plt.axvline(Mean - Std, color="k", linestyle="--")
    plt.axvline(Median, color="g", linestyle="--", label=f"Median={Median:0.2f}")
    plt.axvline(Mode, color="b", linestyle="--", label=f"Mode={Mode:0.2f}")
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    if xlims is not None:
        plt.xlim(xlims)
    plt.title(title)
    plt.legend(loc="upper left")
    plt.savefig(filename)
    plt.show()

Using the above, let's plot the data and see if we can build a reasonable sensor model.

In [ ]:
plot_measurement_statistics(
    bathy_differences,
    "Raw Bathymetry Measurement Statistics",
    "Difference (meters)",
    "Density",
    "bathy_diff_stats_raw.png",
    xlims=(-1000, 1000),
)
print(f"Max: {np.max(bathy_differences[~np.isnan(bathy_differences)])}")

So from the above we can see that the data resembles a normal distribution, but has some substantial outliers. So let's filter out the outliers and then plot the data again.

In [ ]:
# Filter out the outliers
bathy_differences_filtered = bathy_differences[np.abs(bathy_differences - bathy_mean_d) < 3 * bathy_std]
plot_measurement_statistics(
    bathy_differences_filtered,
    "Filtered Bathymetry Measurement Statistics",
    "Difference (meters)",
    "Density",
    "bathy_diff_stats_filtered.png",
    xlims=(-250, 250),
)

With the filtered data we can see that the data is much more normally distributed. We can now use this data to build a sensor model for the particle filter. This model is a simple zero-mean normal distribution with a standard deviation of 100 meters. The statistical mean of the data suggests that there might be a sensor bias, but do to the fact that bathymetric measurements provide a direct measurement of the vertical state, it is unclear if this is an error in the sensor or the state estimate.

--------------------

## Experimental configuration

There are a few questions that can be answered with this simulation:

1. As a general long-term navigation aide - akin to a GPS system - how well does the particle filter perform? This can be answered by running the particle filter with a well-known initial state and tracking the drift error over time.
2. How well does the particle filter perform in a short-term navigation scenario? In other words, given a de-localized INS, how well does the particle filter perform in recovering the true state? This scenario poses a more specific scenario where the platform needs a position fix from GPS but is unable to get one.

I'll analyze both scenarios in this notebook by using two different initial states. The first state will be a well-known state (akin to a GPS fix) and the second will be a de-localized state (delocalized to a drift rate ~1 nmi).

### Performance criteria

The performance of the particle filter will be evaluated based on the following haversine distance error criteria:
1. Overall weighted root-mean-square error of the entire particle cloud (a measure of the overall accuracy of the particle filter)
2. Haversine error of the particle filter estimate (weighted average of all particles)
3. Wasserstein distance between the particle filter estimate and the true state (a measure of the distribution of the particle filter estimate) where the covariance (variance along the diagonal) matrix is derived from the particle field.
These criteria should ultimately be used to evaluate the performance with respect to two concepts: the overall accuracy of the particle filter estimate to recover truth, and with respect to the distance traveled by the platform.

### Truth model

The source data used in this experiment is fundamentally survey data (i.e. measurements that are time stamped and geolocated). I simulate IMU data in order to develop a trajectory, but this isn't specifically recorded IMU data. In order to generate a seperate effective truth, the simulated IMU data is integrated to generate an INS trajectory. This integrated trajectory is then in need of correction. For a truth comparison, I then use the GPS locations of the geophysical measurements to correct the IMU integration to develop a "Truth" trajectory. The same integrated trajectory is then used (through the NED velocities) to propagate the particle filter. The particle filter itself then is corrected using the bathymetric measurements. The error metric are then a comparison of the particle filter's estimate to the INS's "truth" estimate.

In [1]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from pyins import transform
from src.geophysical import gmt_toolbox as gmt

#####
from src.data_management import dbmgr
from src.geophysical import particle_filter as pf
from src.geophysical.gmt_toolbox import GeophysicalMap, MeasurementType, ReliefResolution
import haversine as hs
import h5py
from tqdm import tqdm


def save_simulation_results(
    filename: str, pf_result: pd.DataFrame, trajectory: pd.DataFrame, trajectory_sd: pd.DataFrame
):
    # Create an HDF5 file
    with h5py.File(f"{filename}.h5", "w") as f:
        # Save the result dataframe
        result_group = f.create_group("result")
        for column in pf_result.columns:
            result_group.create_dataset(column, data=pf_result[column].values)

        # Save the feedback.trajectory dataframe
        trajectory_group = f.create_group("trajectory")
        for column in trajectory.columns:
            trajectory_group.create_dataset(column, data=trajectory[column].values)

        # Save the feedback.trajectory_sd dataframe
        trajectory_sd_group = f.create_group("trajectory_sd")
        for column in trajectory_sd.columns:
            trajectory_sd_group.create_dataset(column, data=trajectory_sd[column].values)

In [2]:
db = dbmgr.DatabaseManager(".db")
all_trajs = db.get_all_trajectories()
pf_config = pf.ParticleFilterConfig.from_dict(
    {
        "n": 100,
        "cov": [0.01, 0.01, 1, 0.1, 0.1, 0.1, 0, 0, 0, 0],
        "noise": [0.0, 0.0, 0.0, 1.75, 1.75, 0.01, 0.001, 0.001, 0.001, 0.1],
        "input_config": pf.ParticleFilterInputConfig.VELOCITY,
        "measurement_config": [{"name": "bathymetry", "std": 100}],
    }
)

In [ ]:
bathy_trajectories = all_trajs[all_trajs["depth"]]
bathy_trajectories

In [ ]:
bathy_trajectories.loc[bathy_trajectories["id"] == 900]["duration"] <= 3600

In [11]:
valid_bathy_trajectories = bathy_trajectories[bathy_trajectories["duration"] >= 3600]

In [ ]:
valid_bathy_trajectories.tail()

In [ ]:
db.get_trajectory(108)

In [15]:
# Create the output directory for saving results
output_dir = "bathy_pf_results"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
# Loop over the data
for j in tqdm(bathy_trajectories["id"]):
    if bathy_trajectories.loc[bathy_trajectories.index[j], "duration"] < 3600:
        print(f"Trajectory {j} is too short: {all_trajs.iloc[j]['duration']}")
        continue
    name = db.get_all_trajectories()["source"][j] + "_" + str(j)

    try:
        truth = db.get_trajectory(j)
    except:
        print(f"Failed to get trajectory {j}")
        continue
    try:
        _, feedback = pf.calculate_truth(truth)
        trajectory = feedback.trajectory
        ins_errors = transform.compute_state_difference(truth, trajectory)
    except:
        print(f"Failed to calculate truth for trajectory {j}")
        continue
    distance = truth["distance"].to_numpy()

    min_lat = truth.lat.min()
    max_lat = truth.lat.max()
    min_lon = truth.lon.min()
    max_lon = truth.lon.max()

    try:
        bathy_map = GeophysicalMap(
            MeasurementType.BATHYMETRY, ReliefResolution.FIFTEEN_SECONDS, min_lon, max_lon, min_lat, max_lat, 0.25
        )
    except:
        print(f"Failed to get bathymetry map for trajectory {j}")
        continue
    geomaps = {gmt.MeasurementType.BATHYMETRY: bathy_map}
    try:
        result = pf.run_particle_filter(truth, trajectory, geomaps, pf_config)
    except:
        print(f"Failed to run particle filter for trajectory {j}")
        continue
    try:
        out_path = os.path.join(output_dir, f"{name}")
        save_simulation_results(f"{out_path}", result, trajectory, feedback.trajectory_sd)
    except:
        print(f"Failed to save results for trajectory {j}")
        continue

In [ ]:
plt.contourf(bathy_map.map_data.lon, bathy_map.map_data.lat, bathy_map.map_data, cmap="ocean")
plt.plot(truth.lon, truth.lat, "k.", label="Truth")
plt.plot(trajectory.lon, trajectory.lat, "g.", label="INS")
plt.plot(result.lon, result.lat, "y.", label="PF")
plt.colorbar()
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("2D trajectory")
# plt.xlim(min_lon, max_lon)
# plt.ylim(min_lat, max_lat)
plt.legend()
plt.show()

In [ ]:
hsv = hs.haversine_vector(truth[["lat", "lon"]].to_numpy(), result[["lat", "lon"]].to_numpy(), hs.Unit.METERS)
plt.plot(result.index, result["rms_error_2d"], "b")
plt.plot(result.index, hsv, "g")
plt.plot(result.index, result["estimate_error"], "y")
plt.ylim(0, 3000)
plt.axhline(1852)
plt.axhline(452, color="r")
plt.show()

In [364]:
summary = pd.DataFrame()
error_summary = pd.Series(err, distance)
summary[f"{j}"] = error_summary

In [ ]:
plt.plot(distance, estimate[:, 3:6])

In [ ]:
# Plot the results as a function of distance traveled
plt.plot(
    distance / 1852,
    100 * np.sqrt(ins_errors.north**2 + ins_errors.east**2) / distance,
    "g-",
    label="INS error",
)
plt.plot(
    distance / 1852,
    100 * np.sqrt(integrator_error.north**2 + integrator_error.east**2) / distance,
    "r.",
    label="integrator error",
)
plt.xlabel("Distance traveled (nmi)")
plt.ylabel("Position error (% of distance traveled)")
plt.title("Radial position error vs. distance traveled")
plt.legend()
plt.ylim(0, 100)
plt.show()

# Data Post Processing

Use this section to load and post process the results data set

In [132]:
import os
import numpy as np
import pandas as pd
import haversine as hs
from matplotlib import pyplot as plt
from glob import glob
import h5py
from src.geophysical import gmt_toolbox as gmt
from src.data_management.m77t import find_periods


def load_simulation_results(filename: str) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    # Open the HDF5 file
    with h5py.File(filename, "r") as f:
        # Load the result dataframe
        result_data = {column: f["result"][column][:] for column in f["result"]}
        result = pd.DataFrame(result_data)

        # Load the feedback.trajectory dataframe
        trajectory_data = {column: f["trajectory"][column][:] for column in f["trajectory"]}
        trajectory = pd.DataFrame(trajectory_data)

        # Load the feedback.trajectory_sd dataframe
        trajectory_sd_data = {column: f["trajectory_sd"][column][:] for column in f["trajectory_sd"]}
        trajectory_sd = pd.DataFrame(trajectory_sd_data)

    distance = hs.haversine_vector(
        trajectory[["lat", "lon"]].to_numpy(), trajectory[["lat", "lon"]].shift(1).to_numpy(), hs.Unit.METERS
    )
    distance = np.cumsum(np.nan_to_num(distance, nan=0))

    # time = trajectory.index.to_numpy()

    result["distance"] = distance
    trajectory["distance"] = distance

    lat_lon_var = result[["lat_var", "lon_var"]].to_numpy()
    planar_variance = hs.haversine_vector(np.zeros_like(lat_lon_var), lat_lon_var, hs.Unit.METERS)
    three_d_variance = np.sqrt(planar_variance**2 + result["alt_var"].to_numpy() ** 2)

    result["planar_variance"] = planar_variance
    result["three_d_variance"] = three_d_variance

    # trajectory_sd["distance"] = distance

    return result, trajectory, trajectory_sd


def plot_estimate_error(
    trajectory: pd.DataFrame,
    result: pd.DataFrame,
    drift_rate: np.ndarray,
    map_resolution: float = 452,
    figure_size: tuple[int, int] = (12, 6),
    title_str: str = "Particle Filter Estimate Error",
    recovery_offset: float = 0,
) -> plt.Figure:
    drift_rate += recovery_offset
    fig, axes = plt.subplots(2, 1, figsize=figure_size, gridspec_kw={"height_ratios": [3, 1]})

    duration = trajectory.index[-1] * 60
    axes[0].plot(trajectory["distance"] / 1000, drift_rate, "m", label="INS Drift Rate")
    axes[0].fill_between(
        trajectory["distance"] / 1000,
        result["estimate_error"] - result["planar_variance"] / 2,
        result["estimate_error"] + result["planar_variance"] / 2,
        color="k",
        alpha=0.2,
        label="PF planar certainty",
    )
    axes[0].plot(trajectory["distance"] / 1000, result["estimate_error"], "k", label="Estimate error")

    axes[0].fill_between(
        trajectory["distance"] / 1000,
        np.maximum(result["estimate_error"] + result["planar_variance"] / 2, map_resolution),
        drift_rate,
        where=result["estimate_error"] + result["planar_variance"] / 2 <= drift_rate,
        color="magenta",
        alpha=0.3,
        label="Estimate error less than drift",
    )

    axes[0].fill_between(
        trajectory["distance"] / 1000,
        result["estimate_error"] + result["planar_variance"] / 2,
        map_resolution,
        where=result["estimate_error"] + result["planar_variance"] / 2 <= map_resolution,
        color="green",
        alpha=0.3,
        label="Estimate Error less than map resolution",
    )
    axes[0].axhline(452, color="g", linestyle="--", label="Map resolution")
    axes[0].set_xlabel("Distance traveled (km)")
    axes[0].set_ylabel("Error (meters)")
    axes[0].set_xlim(left=0, right=trajectory["distance"].max() / 1000)
    axes[0].set_ylim(bottom=0)
    axes[0].legend()
    axes[0].set_title(f"{title_str} | Duration: {duration/3600:0.2f} hours")

    # Compress the y-axis of the second plot
    axes[1].set_aspect(aspect="auto", adjustable="datalim")
    axes[1].plot(trajectory["distance"] / 1000, result["planar_variance"], "k", label="2D")
    axes[1].plot(trajectory["distance"] / 1000, result["three_d_variance"], "b", label="3D")
    axes[1].set_xlabel("Distance traveled (km)")
    axes[1].set_ylabel("Estimate Certainty (m)")
    axes[1].set_xlim(left=0, right=trajectory["distance"].max() / 1000)
    axes[1].set_ylim([0, 5000])
    axes[1].legend()
    return fig


def find_estimate_statistic(result: pd.DataFrame, threshold_mask: list | np.ndarray) -> pd.DataFrame:
    """
    Find the periods where the estimate error is below a certain threshold
    """
    position_recovery_periods = find_periods(threshold_mask)
    recoveries = pd.DataFrame()
    for inds in position_recovery_periods:
        start = result.index[inds[0]] * 60
        end = result.index[inds[1]] * 60
        duration = end - start
        distance = result["distance"].iloc[inds[1]] - result["distance"].iloc[inds[0]]
        start_distance = result["distance"].iloc[inds[0]]
        end_distance = result["distance"].iloc[inds[1]]
        mean_error = result["estimate_error"].iloc[inds[0] : inds[1]].mean()
        median_error = result["estimate_error"].iloc[inds[0] : inds[1]].median()
        recovery = pd.DataFrame(
            {
                "start": [start],
                "end": [end],
                "duration": [duration],
                "mean_error": [mean_error],
                "median_error": [median_error],
                "distance": [distance],
                "start_distance": [start_distance],
                "end_distance": [end_distance],
                "distance_traveled": end_distance - start_distance,
            }
        )
        if duration > 0:
            recoveries = pd.concat([recoveries, recovery])

    return recoveries

In [133]:
result_files = glob(os.path.join("scripts", "bathy_pf_results_delocalized", "*.h5"))

In [ ]:
i = 2
result, trajectory, trajectory_std = load_simulation_results(result_files[i])
print(f"file: {result_files[i]}")
fig = plot_estimate_error(
    trajectory,
    result,
    drift_rate=trajectory.index.to_numpy() * 60 * 1852 / (24 * 3600),
    figure_size=(12, 4),
    title_str="De-localized Particle Filter Estimate Error",
    recovery_offset=1852,
)

In [ ]:
estimate_error_summary = pd.DataFrame()
rms_2d_summary = pd.DataFrame()
rms_3d_summary = pd.DataFrame()
distances = pd.DataFrame()
recoveries = pd.DataFrame()
below_pixel = pd.DataFrame()
certain_recoveries = pd.DataFrame()
certain_below_pixel = pd.DataFrame()

for root, dirs, files in os.walk("scripts/bathy_pf_results_delocalized"):
    for file in files:
        if file.endswith(".h5"):
            # filename = os.path.join(root, file).split('.')[0]
            filename = file.split(".")[0]
            result, trajectory, trajectory_std = load_simulation_results(os.path.join(root, f"{filename}.h5"))
            drift_rate = trajectory.index.to_numpy() * 60 * 1852 / (24 * 3600)

            estimate_error_plot = plot_estimate_error(
                trajectory,
                result,
                drift_rate=drift_rate,
                figure_size=(12, 4),
                title_str="De-localized Particle Filter Estimate Error",
                recovery_offset=1852,
            )
            estimate_error_plot.savefig(os.path.join(root, f"{filename}.png"))
            # estimate_error_plot.close()
            plt.close(estimate_error_plot)
            estimate_error = result["estimate_error"].rename(filename)
            estimate_error.index = result["distance"]
            estimate_error_summary = pd.merge(
                estimate_error_summary, estimate_error, left_index=True, right_index=True, how="outer"
            )

            recoveries = pd.concat(
                [recoveries, find_estimate_statistic(result, result["estimate_error"] >= drift_rate)]
            )
            below_pixel = pd.concat([below_pixel, find_estimate_statistic(result, result["estimate_error"] >= 452)])

            certain_recoveries = pd.concat(
                [certain_recoveries, find_estimate_statistic(result, result["planar_variance"] >= drift_rate)]
            )
            certain_below_pixel = pd.concat(
                [below_pixel, find_estimate_statistic(result, result["planar_variance"] >= 452)]
            )

    # rms_2d_summary = pd.concat([rms_2d_summary, result["rms_error_2d"]], axis=1)
    # rms_3d_summary = pd.concat([rms_3d_summary, result["rms_error_3d"]], axis=1)
    # distances = pd.concat([distances, result["distance"]], axis=1)

In [ ]:
certain_recoveries

In [29]:
from datetime import datetime, timedelta

In [ ]:
recoveries

In [142]:
def print_summary_data(data: pd.DataFrame):
    mean_start = timedelta(seconds=data.mean()["start"])
    mean_end = timedelta(seconds=data.mean()["end"])
    mean_duration = timedelta(seconds=data.mean()["duration"])
    print(f"There were a total of {len(data)} position recoveries.")
    print(f"Mean recovery start: {mean_start}")
    print(f"Mean recovery end: {mean_end}")
    print(f"Mean recovery duration: {mean_duration}")

In [ ]:
print_summary_data(recoveries)

In [ ]:
print_summary_data(below_pixel)

In [ ]:
print_summary_data(certain_recoveries)

In [ ]:
certain_recoveries.mean()

The goal of this experiment is to test the particle filter's ability to estimate the true state, either by maintaining an originally accurate estimate, or by recovering from a de-localized state. We are simulating a marine-grade INS with a drift rate of 1 nm per 24 hours, but comparing and propagting this sytem to a true INS value with GPS measurements.

Error metrics we are interested in:

1. Raw particle filter estimate error in meters plus the standard deviation of the particle filter estimate (a measure of overall accuracy and confidence)
2. Particle filter error normalized by the distance traveled by the platform (a measure of the relative error)
3. Particle filter error normalized by drift rate (indicates how much of the error is due to the drift rate, and how much is due to the particle filter) which should demonstrate it's suitability as a long-term navigation aide for position feedback.
4. Particle filter error (possibly normalized as above) as a percentage of map pixel resolution (15 arcseconds, ~452 meters).

Key finding that we want to prove is that the particle filter can estimate the true state to within the resolution of a given map pixel. This is a key finding as it demonstrates the particle filter's ability to provide a position fix in the absence of GPS that is better than the resolution of the map. Alternatively, if we had an arbitrary map where each pixel had a completely unique signature, the measurement model would only be able to report the particle filter's estimate to within the resolution of the map. Subsequent movements and measurments could then be used to refine the estimate. Because geophysical maps and measurements are not unique, the ability of the particle filter to estimate the true state to within the resolution of the map is a key finding that indicates a performance level that approaches a theoretical limit.

### General idea

Consider a grid based map $m$ with entirely unique signatures $m = \left[m_0 \dots m_n\right]$ where $m_i \neq m_j$ for all $i \neq j$. We have a means to measure that signature with perfect accuracy maing the measurement model $p(z | m) = 1$ for $z = m_i$, and zero elsewhere. $\text{bel}(x_t) = \text{bel}(x_{t-1}) * p(z | m)$. To derive a navigation fix we would then take the maximum value of $\text{bel}(x_t)$ and use that as the navigation fix. However, without knowledge of motion, the state space is similarly limited to the grid cell size. With knowledge of motion, this measurement function becomes a lower limit to the amount of error the position estimate can be resolved to.

In [70]:
two_d = trajectory_std[["north", "east"]]
ins_certainty = np.sqrt(
    hs.haversine_vector(two_d.to_numpy(), np.zeros(two_d.shape), hs.Unit.METERS) ** 2 + trajectory_std["down"] ** 2
)

### General Estimate accuracy

### Estimate accuracy w.r.t. distance traveled

In [ ]:
plt.plot(
    trajectory["distance"] / 1000,
    result["estimate_error"].to_numpy() / trajectory["distance"],
    "b",
    label="Relative to distance traveled",
)
# plt.plot(trajectory["distance"] / 1000, result["estimate_error"].to_numpy() / drift_rate, 'r', label="Relative to INS drift rate")
# plt.fill_between(trajectory["distance"] / 1000,
#                  result["estimate_error"] / trajectory["distance"],
#                  452 / trajectory["distance"],
#                  where=result["estimate_error"] <= 452, color='green', alpha=0.3, label='Estimate Error <= 452')
# plt.axhline(452, color='g', linestyle='--', label='Map Resolution (15 arcsec ~= 452m)')
plt.xlabel("Distance traveled (km)")
plt.ylabel("Error (meters)")
plt.xlim(left=0)
plt.ylim(bottom=0)
plt.legend()
plt.title("Particle Filter Estimate Error")
plt.show()

In [ ]:
# check to see if all the tables in results_tables are present in summary["name"] and if not capture the missing tables
missing = []
for table in results_tables:
    if table not in summary["name"].values:
        missing.append(table)

total = len(results_tables)
num_recoveries = total - len(missing)
print(
    f"There are {total} total trajectories. We were able to recover at least one position fix below drift error in {num_recoveries} ({num_recoveries / total :0.4f}) trajectories."
)

In [ ]:
pixel = summary.loc[summary["min error"] <= 452]

# check to see if the tables in pixel are present in summary["name"] and if not capture the missing tables
missing = []
for table in results_tables:
    if table not in pixel["name"].values:
        missing.append(table)
below_pixel_fixes = total - len(missing)

print(
    f"There are {len(pixel)} total below pixel resolution fixes. We were able to achieve at least one position estimate below drift error in {below_pixel_fixes} ({below_pixel_fixes/total :0.4f}) trajectories."
)
print(f"mean duration: {pixel['duration'].mean()} and median duration: {pixel['duration'].median()}")
print(f"mean error: {pixel['min error'].mean()} and median error: {pixel['min error'].median()}")
print(f"minium duration: {pixel['duration'].min()} and maximum duration: {pixel['duration'].max()}")
print(f"minimum error: {pixel['min error'].min()} and maximum error: {pixel['min error'].max()}")
print(f"mean start: {pixel['start'].mean()} and median start: {pixel['start'].median()}")

In [ ]:
summary.sort_values(by="min error").head(3)

In [ ]:
summary.sort_values(by="start").head(3)

In [ ]:
summary.sort_values(by="duration").tail(3)

In [ ]:
summary.sort_values(by="average_error").head(3)

In [ ]:
# Find the line in summary that has the closest to the mean duration
summary.loc[abs(summary["duration"] - summary["duration"].median()) <= timedelta(minutes=1)]

In [ ]:
summary["duration"].median()

In [ ]:
print(
    f"On average we were able to recover a position fix with an mean duration of {summary['duration'].mean()}, median duration of {summary['duration'].median()} and a mean error of {summary['average_error'].mean()} and median error {summary['average_error'].median()}."
)

print(f"Minimum duration {summary['duration'].min()} and maximum duration {summary['duration'].max()}.")
print(f"Minimum error {summary['average_error'].min()} and maximum error {summary['average_error'].max()}.")

In [ ]:
first = summary.loc[summary["num"] == 0]
# first.head()

print(
    f"The first position recover occurs with a mean of {first['start'].mean()} and median {first['start'].median()} after the start of the trajectory."
)
print(
    f"with an mean duration of {first['duration'].mean()}, median duration of {first['duration'].median()} and a mean error of {first['average_error'].mean()} and median error {first['average_error'].median()}."
)

print(f"Minimum duration {first['duration'].min()} and maximum duration {first['duration'].max()}.")
print(f"Minimum error {first['average_error'].min()} and maximum error {first['average_error'].max()}.")

# Gravity

Recreate the above simulation and measurment model development this time with gravity

In [ ]:
from src.gmt_tool import inflate_bounds, get_map_section, get_map_point
import src.process_dataset as pdset
import numpy as np
import json
from matplotlib import pyplot as plt
import os

config = json.load(open("config.json", "r"))
tables = pdset.get_tables(".db/parsed.db")
gravity_tables = [table for table in tables if "_G_" in table]

In [ ]:
d_gravity = np.array([])

for table in gravity_tables:
    data = pdset.table_to_df(".db/parsed.db", table)
    min_lon = data.LON.min()
    max_lon = data.LON.max()
    min_lat = data.LAT.min()
    max_lat = data.LAT.max()
    min_lon, min_lat, max_lon, max_lat = inflate_bounds(min_lon, min_lat, max_lon, max_lat, 0.25)
    gravity_map = get_map_section(min_lon, max_lon, min_lat, max_lat, "gravity", "01m", "temp")
    d_gravity = np.hstack([d_gravity, data["GRAV_ANOM"] - get_map_point(gravity_map, data.LON, data.LAT)])

config["gravity_mean_d"] = np.mean(d_gravity, where=~np.isnan(d_gravity))
config["gravity_std"] = np.std(d_gravity, where=~np.isnan(d_gravity))

if os.path.exists("config.json"):
    # delete the file
    os.remove("config.json")

with open("config.json", "w") as f:
    json.dump(config, f)

plt.hist(d_gravity, bins=100, density=True)
plt.xlim([-50, 75])
plt.xlabel("Gravity Difference (mGal)")
plt.ylabel("Frequency")
plt.title("Gravity Anomaly Difference")
plt.savefig(".db/plots/gravity_diff.png")
plt.show()

# Magnetics

Recreation with magnetics

In [ ]:
from src.gmt_tool import inflate_bounds, get_map_section, get_map_point
import numpy as np
import json

config = json.load(open("config.json", "r"))
tables = pdset.get_tables(".db/parsed.db")
mag_tables = [table for table in tables if "_M_" in table]

df = pdset.table_to_df(".db/parsed.db", mag_tables[0])
df.head()

In [ ]:
d_magnetics = np.array([])

for table in mag_tables:
    data = pdset.table_to_df(".db/parsed.db", table)
    min_lon = data.LON.min()
    max_lon = data.LON.max()
    min_lat = data.LAT.min()
    max_lat = data.LAT.max()
    min_lon, min_lat, max_lon, max_lat = inflate_bounds(min_lon, min_lat, max_lon, max_lat, 0.25)
    mag_map = get_map_section(min_lon, max_lon, min_lat, max_lat, "magnetic", "02m", "temp")
    d_magnetics = np.hstack([d_magnetics, data["MAG_RES"] - get_map_point(mag_map, data.LON, data.LAT)])

config["magnetic_mean_d"] = np.mean(d_magnetics, where=~np.isnan(d_magnetics))
config["magnetic_std"] = np.std(d_magnetics, where=~np.isnan(d_magnetics))

if os.path.exists("config.json"):
    # delete the file
    os.remove("config.json")

with open("config.json", "w") as f:
    json.dump(config, f)

In [ ]:
plt.hist(d_magnetics, bins=100, density=True)
plt.xlim([-500, 500])
plt.xlabel("Magnetic Difference (nT)")
plt.ylabel("Frequency")
plt.title("Magnetic Residual Difference")
plt.savefig(".db/plots/mag_diff.png")
plt.show()